In [1]:
from almanac.strategy.strategy1 import Strategy1

In [2]:
strat = Strategy1(data_path='/home/ornob/Documents/sp500.csv', multiplier=5, capital=100000, risk_target=0.2)

In [3]:
strat.show_stats()

{'ann_mean': 0.050194640091978465, 'ann_std': 0.19549766807317548, 'sharpe_ratio': 0.2567531397519812, 'skew': -3.6687529381837276, 'avg_drawdown': 0.2333119205507957, 'max_drawdown': 1.2141040208288036, 'quant_ratio_lower': 2.2302136448277934, 'quant_ratio_upper': 1.6017921571611629}


In [6]:
returns = strat.get_return()

In [7]:
returns

1982-09-14         NaN
1982-09-15    0.004026
1982-09-16   -0.005261
1982-09-17   -0.007338
1982-09-20    0.006480
                ...   
2022-09-26   -0.012066
2022-09-27   -0.000546
2022-09-28    0.017293
2022-09-29   -0.019406
2022-09-30   -0.004737
Length: 10188, dtype: float64